In [ ]:
import numpy as np
import h5py
from scipy import signal
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
dataset_size = (54,54)
root_path = "//172.16.88.200/private/2221012/MIOnly_FTP_EEG Dataset and OpenBMI Toolbox for Three BCI Paradigms"
fs = 1000
down_rate = 4
ch = 14 #C4

'Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'FC3', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CPz', 'CP4', 'P1', 'P2', 'POz', 'FT9', 'FTT9h', 'TTP7h', 'TP7', 'TPP9h', 'FT10', 'FTT10h', 'TPP8h', 'TP8', 'TPP10h', 'F9', 'F10', 'AF7', 'AF3', 'AF4', 'AF8', 'PO3', 'PO4'

## Preview

In [ ]:
x = np.load(f"{root_path}/prev_dataset/s1_subj1_train_x.npy")
x = x.transpose((1,0,2))[:,:,ch]
y = np.load(f"{root_path}/prev_dataset/s1_subj1_train_y.npy")
labels = ["right" if l == 1 else "left" for l in y]
labels[:3]
x.shape,y.shape

### ダウンサンプリング

In [ ]:
dx = signal.decimate(x, down_rate) #1000hz / 4 にダウンサンプリング
dfs = fs / down_rate

plt.title("fs 1000hz")
plt.plot(np.array(range(4000))/fs,x[0,:])
plt.show()
plt.title("fs 250hz")
plt.plot(np.array(range(1000))/dfs,dx[0,:])

### 標準化

In [ ]:
dx = StandardScaler().fit_transform(dx.T).T

### パワースペクトルに変換

In [ ]:
# パワースペクトルへの変換
freq, px = signal.periodogram(dx, fs=dfs)


print(len(freq))
plt.plot(freq,np.mean(px,axis=0))
plt.xlim((2,30))
plt.ylim((0,0.2))

## まとめて処理(trainのみ)

In [ ]:
for i, size in enumerate(dataset_size):
    for j in range(size):
        _path = f"prev_dataset/s{(i+1)}_subj{j+1}_train"
        x = np.load(f"{root_path}/{_path}_x.npy")
        x = x.transpose((1,0,2))[:,:,ch]
        
        ### 前処理
        x = signal.decimate(x, down_rate) #ダウンサンプリング
        x = StandardScaler().fit_transform(x.T).T #標準化

        _, x = signal.periodogram(x, fs=fs/down_rate) #パワースペクトル
        ###
        y = np.load(f"{root_path}/{_path}_y.npy")
        labels = ["right" if l == 1 else "left" for l in y]
        with h5py.File(f"{root_path}/dataset.h5", "w" if i== 0 and j == 0 else "r+") as h5:
            session_group = h5.require_group("s" + str(i+1))
            for k,label in enumerate(labels):
                local_group = session_group.require_group("./" + label)
                counter = local_group.attrs.get("count")
                counter = 0 if counter is None else counter + 1
                local_group.attrs["count"] = counter
                d = local_group.create_dataset(f"{counter}",(x.shape[1]),data=x[k,:])
                d.attrs["label"] = label
                d.attrs["subj"] = j+1

## セッション統合

In [12]:
with h5py.File(f"{root_path}/dataset.h5", "r+") as h5:
    new_group = h5.create_group("total")
    for s in range(2):
        session = h5[f"s{s+1}"]
        labels = ["right","left"]
        for label in labels:
            local = session[label]
            c = local.attrs["count"]
            for i in range(c):
                data = local[str(i)]
                counter = new_group.attrs.get("count")
                counter = 0 if counter is None else counter + 1
                new_group.attrs["count"] = counter
                d = new_group.create_dataset(f"{counter}",data[()].shape,data=data[()])
                for atkey in data.attrs:
                    d.attrs[atkey] = data.attrs[atkey]
                d.attrs["label"] = label